In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from threading import Thread
import random, time

In [2]:
class HttpRequest():     
    def get(self, url):
        try:
            return self.__request(url)
        except:            
            secends = random.randint(300, 1800)
            print('出现异常，休息一会儿......')
            print(str(secends) + '秒后继续')
            time.sleep(secends)   # 暂停5~30分钟，然后再尝试一次
            try:
                print('重试请求......')
                return self.__request(url)
            except:
                print('出现异常，休息一会儿......')
                print('30分钟后继续')
                time.sleep(1800)   # 暂停30分钟
                return '' 
                
    def __request(self, url):
        response = requests.get(url, timeout=(10, 15)) 
        response.encoding ='gbk'
        return response

In [3]:
def get_url_of_words(req,addr,filename="data/wordlib/words_urls.csv"):
    resp=req.get(addr)
    soup=BeautifulSoup(resp.content,'html.parser',from_encoding="gbk")
    wenku=soup.find('div',class_='cdiv wenk')
    wenku=bytes(wenku.encode("gbk"))
    soup1=BeautifulSoup(wenku,'html.parser',from_encoding="gbk")
    words=soup1.find_all('a',class_='bblue')
    f=open('houseInfo.txt','a+')
    
    word_dict={}
    
    for word in words:
        zhword=word.text.replace(' ',"").replace('\n',"")
        info=":"+word.get('title')
        url="https://shufa.supfree.net/"+word.get("href")
        word_dict['zhword']=zhword
        word_dict['info']=info
        word_dict['url']=url
        df=pd.DataFrame([word_dict])
        df.to_csv(filename,mode='a',index=None,header=False)
        get_word_info(req,url)
        #print(zhword,info,url)
    print(f"成功获取页：{addr} 中的文字及链接信息并写入{filename} 文件")

In [4]:
def get_word_info(req,addr,path='data/wordlib',filename="data/wordlib/words_info.csv"):
    resp=req.get(addr)
    soup=BeautifulSoup(resp.content,'html.parser',from_encoding="gbk")
    fonts=soup.find_all('div',class_='cdiv')
    for font in fonts:
        font_bytes=bytes(font.encode("gbk"))
        soup1=BeautifulSoup(font_bytes,'html.parser',from_encoding="gbk")
        font_title=None
        font_title=soup1.find('span',class_='bgreen botitle18')
        if font_title is None:
            continue #如果当前cdiv标签下找不到字体信息，则继续找下一个cdiv

        font_info=soup1.find_all('li')  #如果有内容，则找该字体下的不同写法

        font_word={}

        for item in font_info:
            item_bytes=bytes(item.encode('gbk'))
            soup2=BeautifulSoup(item_bytes,'html.parser',from_encoding="gbk")

            words_img=soup2.find('img')

            font_word['title']=font_title.text.strip()
            
            info=str(words_img.get('alt')).split('-')
            font_word['word_artist']=info[0].strip()
            font_word['word_font']=info[1].strip()
            font_word['word']=info[2].replace('“','').replace('”','').strip()
            font_word['word_url']=words_img.get('src')
            df=pd.DataFrame([font_word])
            df.to_csv(filename,mode='a',index=None,header=False)
            '''
            print(font_word['word_artist']+
                      font_word['word_font']+
                      font_word['word']+
                      font_word['word_url'].split('/')[-1],font_word['word_url'])
            '''
            img=req.get(font_word['word_url'])
            
            with open(path+'/'+font_word['word']+'_'+
                      font_word['word_font']+'_'+
                      font_word['word_artist']+'_'+
                      font_word['word_url'].split('/')[-1],
                      'wb') as f:
                f.write(img.content)
            
    print(f"成功获取：{addr} 中的文字及链接信息并写入{filename} 文件")
    

In [5]:
def get_wordlib(req,start_page=1,
               end_page=2,
               addr_base='https://shufa.supfree.net/dity.asp?page='):
    for page in range(start_page,end_page+1):
        addr= addr_base+str(page)
        print(f'正在获取第{page}页中{addr}的字体信息\n-------')
        filename="data/wordlib/words_urls.csv"
        get_url_of_words(req,addr,filename)

In [6]:
req = HttpRequest()
def main():
    #单线程执行：
    #get_wordlib(1,40,'https://shufa.supfree.net/dity.asp?page=')
   
    ##如果支持多线程，可以使用下面的方法
    ##对于该网站，只支持两个线程，其他线程会访问拒绝
   
    # 创建 Thread 实例
    t1 = Thread(target=get_wordlib,args=(req,1,20))
    t2 = Thread(target=get_wordlib,args=(req,21,40))
    # t3 = Thread(target=get_wordlib,args=(21,30))
    # t4 = Thread(target=get_wordlib,args=(31,40))
    # 启动线程运行
    t1.start()
    t2.start()
    # t3.start()
    # t4.start()
    # 等待所有线程执行完毕
    t1.join()  # join() 等待线程终止，要不然一直挂起
    t2.join()
    # t3.join()  # join() 等待线程终止，要不然一直挂起
    # t4.join()
 
if __name__=="__main__":
    main()

正在获取第1页中https://shufa.supfree.net/dity.asp?page=1的字体信息
-------正在获取第21页中https://shufa.supfree.net/dity.asp?page=21的字体信息
-------

成功获取：https://shufa.supfree.net/raky.asp?zi=%FDg 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%D2%BB 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%B3%DD 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%F6%B3 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%B6%A1 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%C1%E4 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%C6%DF 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%CD%F2 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.asp?zi=%FD%88 中的文字及链接信息并写入data/wordlib/words_info.csv 文件
成功获取：https://shufa.supfree.net/raky.a